<br>

# DAEE

In [ ]:
#!pip3 install traquitanas --upgrade

In [ ]:
import os
import shutil
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
from shapely.geometry import Point

In [ ]:
from traquitanas import utils

In [ ]:
from paths import *

<br>

# Dados Tabulares

<br>

## Read Data

In [ ]:
latest_file = max(glob.glob(f'{bruto_path}/*', recursive=True), key=os.path.getmtime)
latest_file

In [ ]:
newestFileCreationDate = datetime.utcfromtimestamp(os.path.getctime(latest_file))
newestFileCreationDate

In [ ]:
#
my_file = os.path.join(latest_file)

# ddd
enc = utils.predict_encoding(my_file, n_lines=1000)
enc

In [ ]:
df = pd.read_csv(
    os.path.join(latest_file),
    sep=';',
    skiprows=1,
    #encoding=enc,
    encoding='latin1',
    dtype={
        # Identificação
        #'QTD': 'quantidade',
        #'COD BACIA': 'cod_bacia',
        'NOME RIO/AQUIFERO': 'object',
        'COD RIO/POCO(DAEE)': 'object',    
        'DIST_FOZ(Km)': 'object',

        # Administrativo
        #'USUARIO': 'usuario',    
        #'AUTOS(DAEE)': 'autos_daee',
        #'USO': 'uso',
        #'SEQ': 'seq',
        #'FINALID_USO': 'finalidade_uso',
        #'SITUAC_ADMIN(DAEE)': 'situacao_administrativa',

        # Vazão
        #'VAZAO(M3/H)': 'vazao_m3h',
        #'HORA/DIA': 'hora_dia',
        #'DIA/MES': 'dia_mes',
        #'MES/ANO': 'mes_ano',

        # Coordenada
        'UTM_NORTE(Km)': 'object',
        'UTM_LESTE(Km)': 'object',
        'UTM_MC': 'object',
    }
)

df.info()
df.head()

<br>

## Rename Columns

In [ ]:
# Colunas Originiais
#display(list(df.columns))

# Renomear Colunas
dict_columns = {
    # Identificação
    'QTD': 'quantidade',
    'COD BACIA': 'cod_bacia',
    'NOME RIO/AQUIFERO': 'nome_rio_aquifero',
    'COD RIO/POCO(DAEE)': 'cod_rio_poco',    
    'DIST_FOZ(Km)': 'dist_foz_km',
    
    # Administrativo
    'USUARIO': 'usuario',
    'AUTOS(DAEE)': 'autos_daee',
    'USO': 'uso',
    'SEQ': 'seq',
    'FINALID_USO': 'finalidade_uso',
    'SITUAC_ADMIN(DAEE)': 'situacao_administrativa',
    
    # Vazão
    'VAZAO(M3/H)': 'vazao_m3h',
    'HORA/DIA': 'hora_dia',
    'DIA/MES': 'dia_mes',
    'MES/ANO': 'mes_ano',
    
    # Coordenada
    'UTM_NORTE(Km)': 'utm_norte_km',
    'UTM_LESTE(Km)': 'utm_leste_km',
    'UTM_MC': 'utm_mc',
}

# Rename Columns
df.rename(columns=dict_columns, inplace=True, errors='ignore')
df

<br>

## Fix Columns

In [ ]:
# Drop Columns
df.drop('quantidade', axis=1, inplace=True, errors='ignore')

# Distância da Foz
df['dist_foz_km'] = df['dist_foz_km'].str.replace(',', '').astype(float)
df['dist_foz_km'] = pd.to_numeric(df['dist_foz_km'])

# Vazão
df['vazao_m3h'] = df['vazao_m3h'].str.replace(',', '').astype(float)
df['vazao_m3h'] = pd.to_numeric(df['vazao_m3h'])

# UTM Norte
df['utm_norte_km'] = df['utm_norte_km'].str.replace(',', '').astype(float)
df['utm_norte_km'] = pd.to_numeric(df['utm_norte_km'])
df['utm_norte_m'] = df['utm_norte_km'] * 1000
df['utm_norte_m'] = df['utm_norte_m'].astype(int)

# UTM Leste
df['utm_leste_km'] = df['utm_leste_km'].str.replace(',', '').astype(float)
df['utm_leste_km'] = pd.to_numeric(df['utm_leste_km'])
df['utm_leste_m'] = df['utm_leste_km'] * 1000
df['utm_leste_m'] = df['utm_leste_m'].astype(int)

# UTM MC
df['utm_mc'] = df['utm_mc'].str.replace(',', '').astype(float)
df['utm_mc'] = pd.to_numeric(df['utm_mc'])
df['utm_mc'] = df['utm_mc'].astype(int)

# Results
df

In [ ]:
# Se o dtype for object, strip
for col in df.columns:
    if df[col].dtype == 'O':
        df[col] = df[col].str.strip()
    else:
        pass
    print('>>> {}'.format(col))
    print(list(df[col][0:10]))
    print(' ')

In [ ]:
df.info()
df.head()

In [ ]:
# Lista Colunas
list_columns = list(df.columns)
display(list_columns)

# Lista Colunas
# list_columns = [
#     'cod_bacia',
    #'nome_rio_aquifero',
    #'cod_rio_poco',
    #'dist_foz_km',
    #'usuario',
    #'autos_daee',
    #'uso',
    #'seq',
    #'finalidade_uso',
    #'situacao_administrativa',
    #'vazao_m3h',
    #'hora_dia',
    #'dia_mes',
    #'mes_ano',
    #'utm_norte_km',
    #'utm_leste_km',
    #'utm_mc',
    #'utm_norte_m',
    #'utm_leste_m'
# ]

# Lista Colunas
#for col in df.columns: display(set(df[col]))

<br>

## Erros

In [ ]:
# Erro: Horas por Dia maior que 24
mask = df['hora_dia'] <= 24
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Dias por mês maior que 31
mask = df['dia_mes']  <= 31
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Mês por Ano 12
mask = df['mes_ano']  <= 12
mask =~ mask
df.loc[mask, 'erro_data'] = 'Erro na hora/dia/mês/ano'

# Erro: Coordenadas sem definição
mask = df['utm_leste_km'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Erro: Coordenadas sem definição
mask = df['utm_norte_km'] == 0
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Erro: Coordenadas sem definição de Meridicano Central
mask = (df['utm_mc'] == 45) | (df['utm_mc'] == 51)
mask =~ mask
df.loc[mask, 'erro_coordenada'] = 'Erro na coordenada'

# Results
df

<br>

## Export

In [ ]:
df.to_csv(
    os.path.join(output_path_tab, 'tab_daee.csv'),
    index=False,
)

In [ ]:
# Exclui Problemas
df_temp = df
df_temp = df_temp[df_temp['erro_data'].isna()]
df_temp = df_temp[df_temp['erro_coordenada'].isna()]

# Export Clean
df_temp.to_csv(
    os.path.join(output_path_tab, 'tab_daee_clean.csv'),
    index=False,
)

In [ ]:
# Seleciona Amostra Aleatória
df_temp = df
df_temp = df_temp.sample(n=300)

# Export Clean
df_temp.to_csv(
    os.path.join(output_path_tab, 'tab_daee_exemplo.csv'),
    index=False,
)